In [9]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Step 1: Get unique launch sites from the DataFrame
launch_sites = spacex_df['Launch Site'].unique()

# Step 2: Create a list of dictionaries with 'label' and 'value' keys
dropdown_options = [{'label': 'All Sites', 'value': 'ALL'}]
for site in launch_sites:
    dropdown_options.append({'label': site, 'value': site})

# Step 3: Update the 'options' attribute of the dcc.Dropdown component
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                dcc.Dropdown(id='site-dropdown',
                                             options=dropdown_options,  # Updated options here
                                             value='ALL',
                                             placeholder="Select a Launch Site here",
                                             searchable=True),

                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(
                                    id='payload-slider',
                                    min=0,
                                    max=10000,
                                    step=1000,
                                    marks={i: str(i) for i in range(0, 10001, 1000)},
                                    value=[min_payload, max_payload]
                                ),
                                
                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
    Output('success-pie-chart', 'figure'),
    [Input('site-dropdown', 'value')]
)
def update_pie_chart(selected_site):
    if selected_site == 'ALL':
        filtered_df = spacex_df
        title = 'Total Successful Launches for All Sites'
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == selected_site]
        title = f'Success vs. Failed Launches for {selected_site}'

    # Count the number of successful and failed launches
    success_count = filtered_df[filtered_df['class'] == 1].shape[0]
    failed_count = filtered_df[filtered_df['class'] == 0].shape[0]

    # Create the pie chart
    fig = px.pie(values=[success_count, failed_count],
                 names=['Success', 'Failed'],
                 title=title,
                 hole=0.5)

    return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
    Output('success-payload-scatter-chart', 'figure'),
    [Input('site-dropdown', 'value'),
     Input('payload-slider', 'value')]
)
def update_scatter_chart(selected_site, payload_range):
    if selected_site == 'ALL':
        data = spacex_df
        title = 'Payload vs. Launch Success for All Sites'
    else:
        data = spacex_df[spacex_df['Launch Site'] == selected_site]
        title = f'Payload vs. Launch Success for {selected_site}'
    
    data = data[(data['Payload Mass (kg)'] >= payload_range[0]) & (data['Payload Mass (kg)'] <= payload_range[1])]
    
    fig = px.scatter(data_frame=data, x='Payload Mass (kg)', y='class', color='Booster Version Category', title=title)
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server()